In [28]:
import pandas as pd
C=pd.read_excel('12-transportation-input-1.xlsx',sheet_name='costs',index_col=0)
S=pd.read_excel('12-transportation-input-1.xlsx',sheet_name='supply',index_col=0)['available']
D=pd.read_excel('12-transportation-input-1.xlsx',sheet_name='demand',index_col=0)['needed']
I = ['Little Rock','Urbana']
J = ['Los Angeles','St. Louis','Detroit']
S['Little Rock']

6

In [41]:
# Write your code here
from gurobipy import Model, GRB
mod = Model()
x = mod.addVars(I,J, name = 'X', vtype = GRB.INTEGER)
mod.update()
mod.setObjective(sum(C.loc[i,j]*x[i,j] for i in I for j in J), GRB.MINIMIZE)
for i in I:
    mod.addConstr(sum(x[i,j] for j in J) <= S[i])
for j in J:
    mod.addConstr(sum(x[i,j] for i in I) >= D[j])
mod.write('12-transportation.lp')
!cat 12-transportation.lp

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  250 X[Little_Rock,Los_Angeles] + 150 X[Little_Rock,St._Louis]
   + 200 X[Little_Rock,Detroit] + 300 X[Urbana,Los_Angeles]
   + 200 X[Urbana,St._Louis] + 150 X[Urbana,Detroit]
Subject To
 R0: X[Little_Rock,Los_Angeles] + X[Little_Rock,St._Louis]
   + X[Little_Rock,Detroit] <= 6
 R1: X[Urbana,Los_Angeles] + X[Urbana,St._Louis] + X[Urbana,Detroit] <= 6
 R2: X[Little_Rock,Los_Angeles] + X[Urbana,Los_Angeles] >= 2
 R3: X[Little_Rock,St._Louis] + X[Urbana,St._Louis] >= 5
 R4: X[Little_Rock,Detroit] + X[Urbana,Detroit] >= 4
Bounds
Generals
 X[Little_Rock,Los_Angeles] X[Little_Rock,St._Louis] X[Little_Rock,Detroit]
 X[Urbana,Los_Angeles] X[Urbana,St._Louis] X[Urbana,Detroit]
End


In [38]:
for j in J:
    print(sum(x[i,j] for i in I) >= D[j])

<gurobi.TempConstr: <gurobi.LinExpr: X[Little Rock,Los Angeles] + X[Urbana,Los Angeles]> >= 2>
<gurobi.TempConstr: <gurobi.LinExpr: X[Little Rock,St. Louis] + X[Urbana,St. Louis]> >= 5>
<gurobi.TempConstr: <gurobi.LinExpr: X[Little Rock,Detroit] + X[Urbana,Detroit]> >= 4>


<gurobi.LinExpr: 250.0 X[Little Rock,Los Angeles] + 150.0 X[Little Rock,St. Louis] + 200.0 X[Little Rock,Detroit] + 300.0 X[Urbana,Los Angeles] + 200.0 X[Urbana,St. Louis] + 150.0 X[Urbana,Detroit]>

**b)** Solve the the minimal cost as well as a plan for sending sales people, as in the sample output below.

In [42]:
# Write your code here
mod.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0x558541ce
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 6e+00]
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 1900.0000000

Root relaxation: cutoff, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      1900.00000 1900.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1900 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.900000000000e+

In [43]:
print(f'Minimal cost: {mod.objval}')


Minimal cost: 1900.0


In [49]:
for i in I:
    for j in J:
        print(f'X[{i},{j}] = {x[i,j].X}')

X[Little Rock,Los Angeles] = 2.0
X[Little Rock,St. Louis] = 4.0
X[Little Rock,Detroit] = -0.0
X[Urbana,Los Angeles] = -0.0
X[Urbana,St. Louis] = 1.0
X[Urbana,Detroit] = 4.0
